In [1]:
%pip install -r requirements.txt
%pip uninstall -y tinarm
%pip install -e ../.

  Cloning https://github.com/TinArmEngineering/tinarm.git (to revision 0.1.89) to /tmp/pip-req-build-zrb8x658
  Running command git clone --filter=blob:none --quiet https://github.com/TinArmEngineering/tinarm.git /tmp/pip-req-build-zrb8x658
  Running command git checkout -q 64b54d6b6ae15b805f347ee865df7811d6a8d3a1
  Resolved https://github.com/TinArmEngineering/tinarm.git to commit 64b54d6b6ae15b805f347ee865df7811d6a8d3a1
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for tinarm: filename=tinarm-0.1-py3-none-any.whl size=8398 sha256=d4f544247158bd69c2d6e2d591f9c060de4dbbee30fb41807c04e94641dc582d
  Stored in directory: /tmp/pip-ephem-wheel-cache-5oqzncdv/wheels/00/4c/e3/4b3aa36f9a510bb151617615ac0ee6a290203b67d99510c06c
Successfully built tinarm
  Attempting uninstall: tinarm
    Found existing installation: tinarm 0.1
    Uninstalling tinar

In [2]:
import matplotlib.pyplot as plt
import pint
from tinarm import NameQuantityPair
from tinarm import Quantity 
from tinarm import Job
from tinarm import Machine
from tinarm.api import JOB_STATUS

import tinarm

import logging
import time
import requests
import uuid
import random
import yaml
import pandas as pd
import pint_pandas
import numpy as np

In [3]:

LOGGING_LEVEL = logging.INFO
STATUS_JOB = {value:key for key,value in JOB_STATUS.items()}

In [4]:

### Configure Logging
logger = logging.getLogger()
logger.setLevel(LOGGING_LEVEL)
logger.info(f"tinarm version {tinarm.__version__}")
logger.info(f"pint_pandas version {pint_pandas.__version__}")

q = pint_pandas.PintType.ureg
q.setup_matplotlib()

2024-03-18 16:59:44,309 - NoJobId - INFO - Kebnekaise - 1683099798.py-><module>() - tinarm version 0.1
2024-03-18 16:59:44,310 - NoJobId - INFO - Kebnekaise - 1683099798.py-><module>() - pint_pandas version 0.5


Log in to Tin Arm Engineering's machine solver, and under profile, retrieve your API key. 
Do not commit your API key to a repository, and consider it like a password.  A good way to keep it out of this code base is to use a configuation file, or environment variable. We will want other things cofigured too, so let's use a yaml file for convenience. 

create a `configurations.yaml` file in this directory with the content.
```yaml
api_key: 668952058c490d0a296da0abb966****
root_url: http://server-go:4300
```
But of course replace the number with your api key

You can then add that file to your `.gitignore` file by executing ```echo 'configurations.yaml' >> .gitignore```

```
!echo 'configurations.yaml' >> .gitignore
```

In [5]:
with open("configurations.yaml", "r") as f:
    config = yaml.safe_load(f)

# Stator

In [6]:
stator_parameters = {
    "slot_liner_thikness": 300 * q.um,
    "stator_bore": 8.20 * q.cm,
    "tooth_tip_depth": 1.5 * q.mm,
    "slot_opening": 1.5 * q.mm,
    "tooth_width": 9.8 * q.mm,
    "stator_outer_diameter": 0.136 * q.m,
    "back_iron_thickness": 5.5 * q.mm,
    "stator_internal_radius": 500 * q.um,
    "number_slots": 12 * q.count,
    "tooth_tip_angle": 70 * q.degrees
    }

air_gap_length = 1 * q.mm

## Rotor
Surface mounted Breadloaf magnets

In [7]:
rotor_parameters = {
    "rotor_od": stator_parameters["stator_bore"] - 2 * air_gap_length,
    "rotor_bore": 40 * q.mm,
    "banding_thickness": 0.5 * q.mm,
    "number_poles": 10 * q.count,
    "magnet_thickness": 4.5 * q.millimeter,
    "magnet_pole_arc": 150 * q.degrees,
    "magnet_inset": 0.25 * q.millimeter
    }

## Simulation Parameters

In [8]:
simulation_parameters = {
       "samples_per_electrical_period": 90 * q.count/q.turn,
        "timestep_intervals": 180 * q.count,
        "active_length": 65 * q.mm}


In [9]:
winding_parameters = {
    "symmetry": 2 * q.count,
    "number_phases": 3 * q.count,
    "number_layers": 2 * q.count,
    "coil_span": 1 * q.count,
    "turns_per_coil": 43 * q.count,
    "empty_slots": 0 * q.count,
    "fill_factor": 42 * q.percent
    }

## Operating Point

In [10]:


current_density_grid, angle_grid = np.meshgrid(np.linspace(2, 10, 5) * q.A/q.mm**2, np.linspace(230, 250+45, 5) * q.degrees)
current_density_grid.reshape(-1), angle_grid.reshape(-1)

(array([ 2.,  4.,  6.,  8., 10.,  2.,  4.,  6.,  8., 10.,  2.,  4.,  6.,
         8., 10.,  2.,  4.,  6.,  8., 10.,  2.,  4.,  6.,  8., 10.]) <Unit('ampere / millimeter ** 2')>,
 array([230.  , 230.  , 230.  , 230.  , 230.  , 246.25, 246.25, 246.25,
        246.25, 246.25, 262.5 , 262.5 , 262.5 , 262.5 , 262.5 , 278.75,
        278.75, 278.75, 278.75, 278.75, 295.  , 295.  , 295.  , 295.  ,
        295.  ]) <Unit('degree')>)

In [11]:
operating_points = [{
    'current_density': cd,
    'current_angle': angle,
    "simulated_speed": 2400 * q.rpm,
    } for cd, angle in zip(current_density_grid.reshape(-1), angle_grid.reshape(-1))]



In [12]:
api = tinarm.Api(config["root_url"], config["api_key"], config["org_id"])

2024-03-18 16:59:44,501 - NoJobId - INFO - Kebnekaise - api.py->__init__() - root_url: https://api.build.tinarmengineering.com


In [13]:
m1 = Machine(stator_parameters, rotor_parameters, winding_parameters)

auto_title = Job(m1, operating_points[0], simulation_parameters).generate_title()

In [14]:
'{title}-{current_density:2.4~P}-{current_angle:2.4~P}'.format(title=auto_title, **operating_points[0]).replace(" ", "-")

'dissonant-containments-2.0-A/mm²-230.0-deg'

In [15]:

jobs = [Job(m1, op, simulation_parameters, title='{title}-{current_density:2.4~P}-{current_angle:2.4~P}'.format(title=auto_title, **op).replace(" ", "-")) for op in operating_points]

jobs_mapping = map(lambda x: api.create_job(x), jobs)
jobs_result = list(jobs_mapping)

In [36]:
status = [STATUS_JOB[api.get_job(j['id'])['status']] for j in jobs_result]
print(status)


['Complete', 'Complete', 'Complete', 'Complete', 'Complete', 'Complete', 'Complete', 'Complete', 'Complete', 'Complete', 'Complete', 'Complete', 'Complete', 'Complete', 'Complete', 'Complete', 'Complete', 'Complete', 'Complete', 'Complete', 'Complete', 'Complete', 'Complete', 'Complete', 'Complete']


In [17]:
for j in jobs_result:
    res = api.update_job_status(j['id'], JOB_STATUS['QueuedForMeshing'])
    time.sleep(0.05)
    print(res)

2024-03-18 16:59:59,417 - NoJobId - INFO - Kebnekaise - api.py->update_job_status() - Updating job status: https://api.build.tinarmengineering.com/jobs/65f864f1dde039e1cf9bf0f5/status/10?node_id=None&apikey=eaef0749053f74de7f65d0011c27c76d
{'id': '65f864f1dde039e1cf9bf0f5', 'owner_org_id': '65f864a7dde039e1cf9bf0f3', 'owner_user_id': '65f85b15e4df63dd53eed222', 'creation_date': '2024-03-18T15:59:45Z', 'status': 21, 'title': 'dissonant-containments-2.0-A/mm²-230.0-deg', 'type': 'electromagnetic_spmbrl_fscwseg', 'node_id': 'None', 'data': [{'section': 'operating_point', 'name': 'current_density', 'value': {'magnitude': [2], 'shape': [], 'units': [{'name': 'ampere', 'exponent': 1}, {'name': 'millimeter', 'exponent': -2}]}}, {'section': 'operating_point', 'name': 'current_angle', 'value': {'magnitude': [230], 'shape': [], 'units': [{'name': 'degree', 'exponent': 1}]}}, {'section': 'operating_point', 'name': 'simulated_speed', 'value': {'magnitude': [2400], 'shape': [1], 'units': [{'name': 

example of hashing the input

In [30]:
jobs_result[0]['data'][0]['value']

{'magnitude': [2],
 'shape': [],
 'units': [{'name': 'ampere', 'exponent': 1},
  {'name': 'millimeter', 'exponent': -2}]}

In [31]:
def decode(enc):
    return q.Quantity.from_tuple(
        (enc["magnitude"], ((e["name"], e["exponent"]) for e in enc["units"]))
    )

In [32]:
stator_params = {x['name']:decode(x['value']) for x in jobs_result[0]['data'] if x['section']=='stator'}
stator_params

{'slot_liner_thikness': array([300]) <Unit('micrometer')>,
 'stator_bore': array([8.2]) <Unit('centimeter')>,
 'tooth_tip_depth': array([1.5]) <Unit('millimeter')>,
 'slot_opening': array([1.5]) <Unit('millimeter')>,
 'tooth_width': array([9.8]) <Unit('millimeter')>,
 'stator_outer_diameter': array([0.136]) <Unit('meter')>,
 'back_iron_thickness': array([5.5]) <Unit('millimeter')>,
 'stator_internal_radius': array([500]) <Unit('micrometer')>,
 'number_slots': array([12]) <Unit('count')>,
 'tooth_tip_angle': array([70]) <Unit('degree')>}

In [33]:
base_unit_stator_params = {k:v.to_base_units() for k,v in stator_params.items()}
base_unit_stator_params


{'slot_liner_thikness': array([0.0003]) <Unit('meter')>,
 'stator_bore': array([0.082]) <Unit('meter')>,
 'tooth_tip_depth': array([0.0015]) <Unit('meter')>,
 'slot_opening': array([0.0015]) <Unit('meter')>,
 'tooth_width': array([0.0098]) <Unit('meter')>,
 'stator_outer_diameter': array([0.136]) <Unit('meter')>,
 'back_iron_thickness': array([0.0055]) <Unit('meter')>,
 'stator_internal_radius': array([0.0005]) <Unit('meter')>,
 'number_slots': array([12]) <Unit('count')>,
 'tooth_tip_angle': array([1.22173048]) <Unit('radian')>}

In [34]:
int_per_unit_stator_params = base_unit_stator_params.copy()
for key in int_per_unit_stator_params:
    if int_per_unit_stator_params[key].dimensionality is base_unit_stator_params['stator_bore'].dimensionality:
        int_per_unit_stator_params[key] = int((base_unit_stator_params[key]/base_unit_stator_params['stator_bore']).magnitude * 2**15) 
    elif int_per_unit_stator_params[key].dimensionality is base_unit_stator_params['tooth_tip_angle'].dimensionality:
        int_per_unit_stator_params[key] = int((base_unit_stator_params[key]/np.pi).magnitude * 2**15)
    else:
        int_per_unit_stator_params[key] = int(base_unit_stator_params[key].magnitude)

/tmp/ipykernel_85424/1904372677.py:4: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  int_per_unit_stator_params[key] = int((base_unit_stator_params[key]/base_unit_stator_params['stator_bore']).magnitude * 2**15)
/tmp/ipykernel_85424/1904372677.py:8: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  int_per_unit_stator_params[key] = int(base_unit_stator_params[key].magnitude)
/tmp/ipykernel_85424/1904372677.py:6: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  int_per_unit

In [35]:
per_unit_stator_params = base_unit_stator_params.copy()
for key in per_unit_stator_params:
    if per_unit_stator_params[key].dimensionality is base_unit_stator_params['stator_bore'].dimensionality:
        per_unit_stator_params[key] = (base_unit_stator_params[key]/base_unit_stator_params['stator_bore']).magnitude 
    elif per_unit_stator_params[key].dimensionality is base_unit_stator_params['tooth_tip_angle'].dimensionality:
        per_unit_stator_params[key] = (base_unit_stator_params[key]/np.pi).magnitude 
    else:
        per_unit_stator_params[key] = base_unit_stator_params[key].magnitude

In [24]:
base_unit_stator_params

{'slot_liner_thikness': array([0.0003]) <Unit('meter')>,
 'stator_bore': array([0.082]) <Unit('meter')>,
 'tooth_tip_depth': array([0.0015]) <Unit('meter')>,
 'slot_opening': array([0.0015]) <Unit('meter')>,
 'tooth_width': array([0.0098]) <Unit('meter')>,
 'stator_outer_diameter': array([0.136]) <Unit('meter')>,
 'back_iron_thickness': array([0.0055]) <Unit('meter')>,
 'stator_internal_radius': array([0.0005]) <Unit('meter')>,
 'number_slots': array([12]) <Unit('count')>,
 'tooth_tip_angle': array([1.22173048]) <Unit('radian')>}

In [25]:
per_unit_stator_params

{'slot_liner_thikness': array([0.00365854]),
 'stator_bore': array([1.]),
 'tooth_tip_depth': array([0.01829268]),
 'slot_opening': array([0.01829268]),
 'tooth_width': array([0.1195122]),
 'stator_outer_diameter': array([1.65853659]),
 'back_iron_thickness': array([0.06707317]),
 'stator_internal_radius': array([0.00609756]),
 'number_slots': array([12]),
 'tooth_tip_angle': array([0.38888889])}

In [26]:
import hashlib

In [27]:
hashlib.sha256(str(per_unit_stator_params).encode()).hexdigest()

'c31cf65be4cf35096b76907016c1d95db94f21cb8ea64c97605a73e780fbe98b'

In [28]:
hashlib.sha256(str(base_unit_stator_params).encode()).hexdigest()

'b06a5e2fcca81505a28fe3ff11b0e5f24785c3e1b8e9ad26fafcb9e70b833677'

In [29]:
str(base_unit_stator_params)

"{'slot_liner_thikness': <Quantity([0.0003], 'meter')>, 'stator_bore': <Quantity([0.082], 'meter')>, 'tooth_tip_depth': <Quantity([0.0015], 'meter')>, 'slot_opening': <Quantity([0.0015], 'meter')>, 'tooth_width': <Quantity([0.0098], 'meter')>, 'stator_outer_diameter': <Quantity([0.136], 'meter')>, 'back_iron_thickness': <Quantity([0.0055], 'meter')>, 'stator_internal_radius': <Quantity([0.0005], 'meter')>, 'number_slots': <Quantity([12], 'count')>, 'tooth_tip_angle': <Quantity([1.22173048], 'radian')>}"